In [8]:
import numpy as np
import h5py

import info9

In [9]:
info9.read_hdf5_dataset

<function info9.PyCapsule.read_hdf5_dataset>

In [28]:
fname = "testfile.hdf5"
data = np.array([4.,2.,3.])
with h5py.File(fname, "w") as f:
    f.create_dataset("data", data=data)
data=0

Les données sont lues par la librairie C++ : les tableaux seront gérés par la librairie sous la forme de matrices `Eigen::MatrixXd` (par la librairie `Eigen`). Cette librairie a été choisie car `pybind11`, utilisé pour donner des *bindings* Python à la librairie C++, permet d'obtenir une vue des matrices `Eigen::MatrixXd` sous la forme de `numpy.array` dans le code Python, i.e. sans copie !

Le chemin inverse (`numpy.array` vers `Eigen::MatrixXd`) est moins facile, et nécessite une copie. D'autres méthodes permettraient d'obtenir une vue du `numpy.array` depuis la librairie C++, mais elles sont plus intrusives ; la méthode choisie ici permet d'écrire une librairie C++ ne gérant que des matrices `Eigen::MatrixXd` (i.e. sont connaissance de Python), puis de créer des *bindings* donnant une librairie Python très ergonomique.

In [29]:
u = info9.read_hdf5_dataset(fname,"data")
# flags.owndata == false assure que la mémoire est gérée par la
# librairie C++, et qu'on y accède sans copie.
u.flags.owndata

False

In [30]:
# On peut utiliser les fonctions de la librairie sans copie
v = info9.addm(u,u)
v.flags.owndata

False

In [31]:
# On peut également effectué des opérations attendant un numpy.array sur le tableau u,
# mais, si elle n'opère pas en-place, le résultat sera un numpy.array géré par Python.
w = u + u
w.flags.owndata

True

In [36]:
# Les fonctions en-place n'opèrent pas de copie.
u.sort()
u.flags.owndata

False